In [65]:
import requests
import socket

In [20]:
model="llama3.2:1b"

In [116]:
dummy_ip = "46.189.45.146"

## Define tools functions

In [133]:
def get_local_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_address = s.getsockname()[0]
        s.close()
        return ip_address
    except socket.error as e:
        print(f"Error: {e}")
        return None

# Example usage:
local_ip = get_local_ip()
if local_ip:
    print(f"Local IP address: {local_ip}")

# this fails in Jupyter so I'll cheat
local_ip = dummy_ip

Local IP address: 10.5.49.56


In [118]:
def get_location_by_ip(ip_address):
    try:
        response = requests.get(f"http://ip-api.com/json/{ip_address}")
        response.raise_for_status()
        data = response.json()

        return data
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [110]:
def get_city(ip_address="": str) -> str:
    if not ip_address:
        ip_address = get_local_ip()

    location = get_location_by_ip(ip_address)
    return location["city"]

In [119]:
weather_format = "Current+weather+for+%l:+%C,+Temperature:+%t,+Feels+like:+%f,+Precipitation+(mm, next+3+hours):+%p,+Wind:+%w"

def get_current_weather(location: str = "") -> str:
    """
    Get current weather forecast
    Args:
        location (str): The location for which to retrieve the weather forecast.
            This can be a city name, a country name, or a specific geographic coordinate.

    Returns:
        str: The current weather forecast for the specified location, formatted as a string.

    Notes:
        This function uses the wttr.in API to retrieve the weather forecast.
        The weather format is predefined and includes the current weather conditions,
        temperature, feels-like temperature, precipitation, and wind.
    """
    if not location:
        ip_address = get_local_ip()
        location = get_city(dummy_ip)
        
    url = f'http://wttr.in/{location}?format="{weather_format}"'
    response = requests.get(url)
    # print(response.text)

    return response.text

In [111]:
get_city("46.189.45.146")

'Berlin'

In [120]:
get_current_weather()

"Current weather for Berlin: Partly cloudy, Temperature: +11°C, Feels like: +9°C, Precipitation (mm, next 3 hours): 0.0mm, Wind: ↑23km/h"


'"Current weather for Berlin: Partly cloudy, Temperature: +11°C, Feels like: +9°C, Precipitation (mm, next 3 hours): 0.0mm, Wind: ↑23km/h"'

In [136]:
def make_friendly(string: str = "", model:str=model):
    """
    Make a text string less terse, and more user-friendly for end-user to read
    """
    messages = [
    {"role": "user", "content": f"Your task is to present this information in a human-readable, friendly manner: {string}. Don't include any preamble about the task. If it's a weather forecast, do it in the style of an eighties radio DJ giving a weather update"}
    ]
    response = ollama.chat(
    model=model,
    messages=messages,
    )

    return response

## Add tools

In [146]:
tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather for a city",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The name of the city",
                        },
                    },
                    "required": ["city"],
                },
            },
        },
    {
            "type": "function",
            "function": {
                "name": "get_city",
                "description": "Get the city for a given IP address",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ip_address": {
                            "type": "string",
                            "description": "The IP address to get the city for",
                            "default": "",
                        },
                    },
                    "required": [],
                },
            },
        },
        {
        "type": "function",
        "function": {
            "name": "make_friendly",
            "description": "Make a text string less terse, and more user-friendly for end-user to read",
            "parameters": {
                "type": "object",
                "properties": {
                    "string": {
                        "type": "string",
                        "description": "The input text to be made more friendly"
                    },
                    "model": {
                        "type": "string",
                        "description": "The name of the language model to use for generating the friendly text",
                        "default": model
                    }
                },
                "required": []
            }
        }
    }
    ]

## Extract function

In [152]:
import ollama

def get_response(prompt: str, tools: list=tools) -> dict:
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        tools=tools,
    )

    return response

In [153]:
response = get_response(f"Give a friendly weather forecast for {location['city']}? Return your answer in a friendly manner")

In [154]:
response

ChatResponse(model='llama3.2:1b', created_at='2025-01-27T15:00:44.850516787Z', done=True, done_reason='stop', total_duration=958407908, load_duration=56230482, prompt_eval_count=312, prompt_eval_duration=59000000, eval_count=18, eval_duration=841000000, message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_current_weather', arguments={'city': 'Berlin'}))]))

### Only first function for now for testing

In [129]:
func_name = response.message.tool_calls[0].function.name

In [130]:
if func_name in globals():
    func = globals()[func_name]
else:
    print("Can't find function")

In [131]:
params = response.message.tool_calls[0].function.arguments

In [132]:
func_output = func(params)

"Current weather for  'Berlin'}: Partly cloudy, Temperature: +11°C, Feels like: +9°C, Precipitation (mm, next 3 hours): 0.0mm, Wind: ↑23km/h"


## Make friendly and readable

In [138]:
get_response(f"Get the user's city then find nearby events")

ChatResponse(model='llama3.2:1b', created_at='2025-01-27T14:56:39.189643461Z', done=True, done_reason='stop', total_duration=17870736103, load_duration=1371770478, prompt_eval_count=222, prompt_eval_duration=4734000000, eval_count=222, eval_duration=11762000000, message=Message(role='assistant', content='{\n\n"get_current_weather": {\n"city": "New York"\n},\n\n"nearby_events": [\n{\n"event_name": "Fireworks at Times Square",\n"date": "2024-03-15T20:00:00+00:00",\n"location": "New York, NY",\n"organizer": "The City of New York",\n"distance_from_user_location": 10.5\n},\n{\n"event_name": "Beach Festival at Ocean Park",\n"date": "2024-03-15T14:00:00+00:00",\n"location": "Los Angeles, CA",\n"organizer": "City of Los Angeles",\n"distance_from_user_location": 22.1\n},\n{\n"event_name": "Stadium Show at Dodger Stadium",\n"date": "2024-03-15T18:00:00+00:00",\n"location": "Los Angeles, CA",\n"organizer": "Los Angeles Convention and Visitors Bureau",\n"distance_from_user_location": 21.6\n}\n]\n\

In [106]:
# messages = [
#     {"role": "user", "content": f"Your task is to present this information in a human-readable, friendly manner: {func_output}. Don't include any preamble about the task. If it's a weather forecast, do it in the style of an eighties radio DJ giving a weather update"}
# ]

In [107]:
# response = ollama.chat(
#     model=model,
#     messages=messages,
#     # tools=tools,
# )

In [109]:
forecast = response.message.content

'"Alright, Berlin fans! Check out this sweet forecast for you. It\'s partly cloudy tonight, with temps rising to +11°C and feeling like +9°C - that\'s right, it\'s gonna feel like 5 degrees warmer than the actual temp. Now, I know what you\'re thinking... precipitation? Well, let me tell you, we\'re looking at a good amount of that in the next few hours, with just 0.0mm showing up in the first three frames. And as for the wind, it\'s gonna be blowing at +23km/h - that\'s pretty strong, if I do say so myself.\n\nStay cool out there, Berlin! Get your sunglasses ready and your coffee warm."'

## Read it out

In [157]:
import os

os.environ["REPLICATE_API_TOKEN"] = "r8_InimumOLV1btQl6aH4IylyYV2OwnlOm2t5pem"

In [160]:
text = '"Alright, Berlin fans! Check out this sweet forecast for you. It\'s partly cloudy tonight, with temps rising to +11°C and feeling like +9°C - that\'s right, it\'s gonna feel like 5 degrees warmer than the actual temp. Now, I know what you\'re thinking... precipitation? Well, let me tell you, we\'re looking at a good amount of that in the next few hours, with just 0.0mm showing up in the first three frames. And as for the wind, it\'s gonna be blowing at +23km/h - that\'s pretty strong, if I do say so myself.\n\nStay cool out there, Berlin! Get your sunglasses ready and your coffee warm."'

In [159]:
import replicate

def read_aloud(text: str):
    output = replicate.run(
        "platform-kit/mars5-tts:6aed0f11f3ba7b13d59ab3228355e7b1ea943479673cc57e10e99ba766536811",
        input={
            "text": text,
            "top_k": 1,
            "temperature": 1.1,
            "freq_penalty": 5,
            "ref_audio_file": "https://replicate.delivery/pbxt/L9a6SelzU0B2DIWeNpkNR0CKForWSbkswoUP69L0NLjLswVV/voice_sample.wav",
            "rep_penalty_window": 150,
            "ref_audio_transcript": "Hi there. I'm your new voice clone. Try your best to upload quality audio."
        }
    )

    return output
    # print(output)

In [162]:
speech = read_aloud(text)

In [163]:
output

In [164]:
output.url

'https://replicate.delivery/yhqm/lvrNQSNfGLU7JKYrWFuFzCx5elUnLPP2fNB52feGU4DHQtKhC/output.mp3'

In [168]:
!ls

main.py  ollama_tools.ipynb  ollama-tools.py  output.mp3  _posts  raccoon.jpg


In [167]:
!wget {output.url}

--2025-01-27 16:41:25--  https://replicate.delivery/yhqm/lvrNQSNfGLU7JKYrWFuFzCx5elUnLPP2fNB52feGU4DHQtKhC/output.mp3
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving replicate.delivery (replicate.delivery)... 34.107.252.52, 2600:1901:0:8c88::
Connecting to replicate.delivery (replicate.delivery)|34.107.252.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20589 (20K) [audio/mpeg]
Saving to: ‘output.mp3’

output.mp3          100%[===================>]  20.11K  --.-KB/s    in 0.01s   

2025-01-27 16:41:25 (1.89 MB/s) - ‘output.mp3’ saved [20589/20589]



In [169]:
filename = output.url.split("/")[-1]

In [170]:
filename

'output.mp3'

In [171]:
!aplay {filename}

Playing raw data 'output.mp3' : Unsigned 8 bit, Rate 8000 Hz, Mono
